# Aligning reads & processing SAM files

Exercise for creating and processing short read alignments.

* **Contact:** mate.balajti@unibas.ch

### General

Be sure to nicely format your answer.
Indicate your name in the file name!
_Ex2_solutions_Name_LastName.py_ would be a good approach to this.
Document, report and detail your work, make sure we can follow and execute it.
If we cannot understand the chaos in your answers, it won't be considered for grading. 

## Prerequisites

### Operating system

For solving these exercises, you need to work in Bash on a Unix-type terminal,
so if you are behind either a Linux distribution, a Mac or a BSD system etc.,
you ​ should ​ be good to go. If instead you only have access to a Windows 10/11
computer, you should be able to make use of the support the system offers for
Linux/Ubuntu through their cooperation with Canonical. For older Windows
versions or even better compatibility, you can also always run a virtual
machine with any Linux distribution you like.

See
[here](https://www.hpe.com/us/en/insights/articles/3-good-ways-to-run-linux-on-windows-2007.html​)
for some pointers for either approach.

Finally, if you have experience with Docker, one other possible option is to
use a Docker image that contains all required software. For example, starting
from a Linux image (e.g., latest Ubuntu), which provides Linux/GNU/Bash out of
the box, you could install STAR and/or any other required Linux software by
writing an appropriate Dockerfile and then building that image. You can also
search online for available images that already contain these tools (e.g.,
[this](https://hub.docker.com/r/mgibio/star/dockerfile​) looks like one you
could use for running STAR; see below). If you decide to go this route, be
aware that Docker on Windows still runs in a VM and support is not always
stable.

> **Note:** If you are planning to do more bioinformatics work in the future,
> it is definitely a good idea to have a stable Linux system at hand at all
> times. In this case, you might want to consider installing a Linux
> distribution side-by-side with your Windows OS (search for “Linux Windows
> dual boot” or similar).

### Software

In the last exercise, you were asked to write a simple, naive short read
aligner from scratch. While this is a good exercise, it is not suitable for
actual analysis, because your code won’t be optimized to handle the amounts of
data that next-generation sequencing typically yields. Since the dawn of
high-throughput sequencing techniques in the mid 2000’s, a lot of effort has
been put into designing and implementing very efficient methods at mapping
short reads to reference sequences. For this exercise, we will use a popular
option called [STAR](https://github.com/alexdobin/STAR). Among many other
features, STAR supports spliced alignments and optionally accepts gene
annotations in GTF format next to a genome reference to increase the fidelity
of mapping reads that cover splice junctions (if not provided, STAR, by
default, will try to infer splice junctions from the genome reference and the
reads). Please either install STAR (and any other third-party software you
need) via the [Conda](https://docs.conda.io/en/latest/miniconda.html) package
manager _OR_ use Docker containers as suggested above.

### Installation for Windows

_STAR_ is only available as Docker image on Windows.
Here we assume Docker is not yet installed and Windows 10 or 11 is used.
Coarse installation steps:
* Install WSL2 (Windows subsystem for Linux) backend
  https://learn.microsoft.com/en-us/windows/wsl/install
* Install Docker https://docs.docker.com/desktop/install/windows-install/
* Ensure the installation was successful by following the _Quick Start Guide_.
* Get the STAR docker image by executing the following command in
a PowerShell or Windows Command Prompt window (in Linux it is called a terminal window).
  ```bash
  docker pull mgibio/star
  ```

> Note: please refer to the most up-to-date documentation
> by checking the online instructions!

### Run a Docker image

To have the local files visible in the container, one needs to mount the directory onto the host (in this case the STAR container). 

For example, to mount `C:\paths\to\test_files` (absolute path to directory `test_files`) onto `/docker_main/files` (the path and name of the directory on the Linux-based host):
```bash
docker run -it \
  --mount type=bind,source="C:\paths\to\test_files",target=/docker_main/files \
  mgibio/star:latest
```

> Note: on Linux, paths are constructed with slash "/", whereas on Windows with backslash "\\"!

> Note: to get help about the command `docker run`,
> try running `docker run --help`
> or search online for the command.

#### Exercise 2.1: Create index (2 points)

Follow STAR’s manual to create an index of the provided genome ​`FASTA`​ file
and ​`GTF`​ gene annotations. Note that to allow the mapping to be done on a
laptop, only chromosome 19 and the corresponding gene annotations are provided.
In a typical setting, indexing and mapping would be done on all chromosomes and
an unfiltered set of annotations. Note also that files are provided in a
compressed form (`GZIP`) for easy sharing and may need to be uncompressed
before use (check the instructions whether passing `GZIP`ped files directly is
accepted).

Required files:

* Genome:​ `Mus_musculus.GRCm38.dna_rm.chr19.fa.gz`
* Gene annotations: `Mus_musculus.GRCm38.88.chr19.gtf.gz`


In [10]:
# gunzip Mus_musculus.GRCm38.dna_rm.chr19.fa.gz 
# gunzip Mus_musculus.GRCm38.88.chr19.gtf.gz
# Uncompress the files

#### Exercise 2.2: Align reads (2 points)

Follow STAR’s manual to align reads to the reference, using the index you have
created in exercise 2.1. Note that we are dealing here with (part of) a
paired-end sequencing library, so you will need to provide both read library
files for this step.

ATTENTION: The running of the mapping may take quite some time if you're working on your local machine. (up to 10-15min)

Required files:

* Control mate 1: `control.mate_1.fq.gz`
* Control mate 2: `control.mate_2.fq.gz`

### Part 2: Process alignments

#### Exercise 2.3: Count reads (3 points)

Find out from the STAR output (log or `SAM` files): (see file `bash_basics.sh` for help)

* How many alignments were reported?
* How many reads were uniquely mapped?  
  **Hint:** check for the `NH` (number of hits) SAM tag
* How many reads were mapped to multiple loci?
* How many reads could not be mapped?

Compare the sum of uniquely mapped, multi-mapped and unmapped reads to the
total number of reads in the ​ FASTQ​ input files. Do the numbers match?

> **Note:** See the [SAM​
> specification](https://samtools.github.io/hts-specs/SAMv1.pdf) for more info
> on SAM files.

#### Exercise 2.4: Run custom functions on STAR results (3 points)

`FASTQ`​ (for reads), ​`FASTA`​ (for reference sequences and reads if sequencing
quality scores are disregarded or discarded), ​`GTF`​/`GFF`​ (for gene
annotations/features), `SAM` (for alignments; with the corresponding
binary/compressed versions ​`BAM`,​ and, more recently, ​`CRAM​`), ​`BED​` (generic
tabular format for representing genomic ranges) are the main file types that
are used in the analysis of RNA-Seq data. Relevant tools in the field will
nowadays almost always require input files and report their own outputs in any
of these formats. However, not every tool accepting a `FASTA`​ file as input
will also accept a ​`FASTQ`​ file, although it is trivial to convert ​`FASTQ​` to
`FASTA` in a non-lossy way. In addition, both for legacy and new tools, custom
formats are still being used, occasionally, to represent specialized
information. Therefore, writing and applying parsers and converters to convert
outputs of one tool such that they can be used as inputs to another is a
somewhat menial, but common task that bioinformaticians are often faced with.

To practice this and connect to the work you have done in the previous exercise:

* Write code that converts a ​`SAM​` file to a ​`FASTA`​ file and apply it on the
  output of exercise 2.2 (align reads). If you didn’t manage to solve exercise 2.3, write
  code that converts ​`FASTQ`​ to ​`FASTA` instead.
* Convert your files to ​`FASTA`​ and then apply your functions from the previous
  session to the output.
  * Note: `map_reads()` will likely not finish in reasonable time.
    Thus, it is fine to stop it. But report and reason about why this could be the case. What makes the difference?

In [3]:
def sam_to_fasta(sam_file, fasta_file):
    with open(sam_file, 'r') as sam, open(fasta_file, 'w') as fasta:
        for line in sam:
            if line.startswith('@'):
                continue
            parts = line.strip().split('\t')
            read_name = parts[0]
            sequence = parts[9]

            fasta.write(f">{read_name}\n{sequence}\n")
    print(f"Fasta file '{fasta_file}' created.")


sam_file = "output_files/control_alignment_Aligned.out.sam"
fasta_file = "reads.fasta"
sam_to_fasta(sam_file, fasta_file)


# It gives me FileNotFoundError, but the file is present and the name is correct.

Fasta file 'reads.fasta' created.


#### Part 3: Differential gene expression analysis (optional)

One of the most common experiment types making use of RNA-Seq is a differential
gene expression analysis where gene expression levels across different
conditions (e.g., healthy vs disease) or compared to each other. This type of
experiment is often the basis for discovering genes that are relevant to a
given physiological or disease process. Following this relatively open-ended
discovery phase of a study, a list of genes with a strikingly different
expression pattern is often the basis for further, more detailed mechanistic
studies.  While performing an entire differential gene expression analysis is
beyond the scope of this limited seminar, you are invited to dig further into
this topic in an open-ended way, by reading on the following topics and either
writing your own code or apply available tools/packages for these tasks:

Useful files:

* Control mate 1: `control.mate_1.fq.gz`
* Control mate 2: `control.mate_2.fq.gz`
* Treated mate 1: `treated.mate_1.fq.gz`
* Treated mate 2: `treated.mate_2.fq.gz`

Procedure:

* Create a table of counts for each gene (rows) and sample (columns)  
  **Hint:** You can easily implement this yourself, but you can also make use
  of the [HTSeq](https://htseq.readthedocs.io/en/release_0.11.1/count.html)
  package for this task (can also be used to report counts for other features)
* Analyze gene expression across different conditions based on such a count
  table  
  **Hint:** You can do this, e.g., with the
  [edgeR](https://www.bioconductor.org/packages/release/bioc/html/edgeR.html)
  package, written in R.

If you are doing any work in this regard, feel free to send it to the teaching assistant(s) for some feedback.

> **Note:** As an alternative to alignment- and count-based approaches to a
> differential gene expression, nowadays probabilistic methods are increasingly
> being used for similar purposes.  These have several advantages, such as
> (typically) requiring fewer steps and less compute resources. Importantly,
> they provide abundances for individual transcripts and thus enable analyses,
> such as differential transcript expression and isoform usage analysis, which
> the count-based methods do not readily allow. A downside for these methods is
> that they are not easy to understand in detail, making results harder to
> interpret and potentially more sensitive to biases. The two main tools for
> alignment-free estimation of transcript abundances are
> [Salmon](​https://github.com/COMBINE-lab/salmon) and
> [kallisto](https://github.com/pachterlab/kallisto​).